In [7]:
import urllib
import json
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import random
from datetime import datetime

In [2]:
lb_url = 'http://www.golfchannel.com/tours/pga-tour/2017/open-championship/'
r = requests.get(lb_url)
soup = BeautifulSoup(r.content, 'html.parser')

stuff = soup.findAll('script')[25].get_text().replace('\n    var tournamentJSON = ', '') \
                .replace(';\n  $(\'body\').data(\'tournamentJSON\', tournamentJSON);\n', '')
data = json.loads(stuff) 

In [4]:
scores = data['scoreboard']['players']
# read in current teetime sheet to add to tee times
teetimes = []
for p_num in scores:
    new = {}
    new['Round'] = 1
    new['Player.Name'] = scores[p_num]['firstname'] + ' ' + scores[p_num]['lastname']
    new['tee_time'] = scores[p_num]['scores']['TeeTime']
    teetimes.append(new)

In [5]:
teetimes = pd.DataFrame(teetimes)

In [9]:
tee_times = teetimes

In [10]:
for time in tee_times['tee_time'].unique():
    tee_times.loc[tee_times['tee_time'] == time, 'tee_time'] = datetime.strptime(time,'%I:%M %p')

In [19]:
tee_times['hour'] = tee_times['tee_time'].astype(str).str.slice(11,16).str.replace(':', '').astype(int)
tee_times = tee_times.sort_values('hour', ascending=True)

In [25]:
tee_times['morning'] = 0
mid_time = tee_times['hour'].median()
tee_times.loc[tee_times['hour'] < mid_time, 'morning'] = 1

In [31]:
tee_times2 = tee_times.copy()

In [32]:
tee_times2['Round'] = 2
tee_times2['morning'] = 1 - tee_times['morning']
tee_times = tee_times.append(tee_times2)

In [34]:
tee_times[tee_times['Player.Name'].str.contains('Moore')]

Player.Name  Round             tee_time  hour  morning
43  Ryan Moore      1  1900-01-01 01:35:00   135        1
43  Ryan Moore      2  1900-01-01 01:35:00   135        0

In [35]:
teetimes.to_csv('teetime_data.csv', index=False)